In [1]:
# import libraries

import numpy as np
import pandas as pd
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

In [2]:
# read data

df = pd.read_excel('Test_set.xlsx', engine='openpyxl')
pd.set_option('display.max_columns', None)

In [3]:
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info
...,...,...,...,...,...,...,...,...,...,...
2666,Air India,6/06/2019,Kolkata,Banglore,CCU → DEL → BLR,20:30,20:25 07 Jun,23h 55m,1 stop,No info
2667,IndiGo,27/03/2019,Kolkata,Banglore,CCU → BLR,14:20,16:55,2h 35m,non-stop,No info
2668,Jet Airways,6/03/2019,Delhi,Cochin,DEL → BOM → COK,21:50,04:25 07 Mar,6h 35m,1 stop,No info
2669,Air India,6/03/2019,Delhi,Cochin,DEL → BOM → COK,04:00,19:15,15h 15m,1 stop,No info


In [4]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

In [5]:
for i in df.columns:
    if df[i].dtype=='O':
        print(df[i].value_counts())
        print('_'*25)

Jet Airways                          897
IndiGo                               511
Air India                            440
Multiple carriers                    347
SpiceJet                             208
Vistara                              129
Air Asia                              86
GoAir                                 46
Multiple carriers Premium economy      3
Vistara Premium economy                2
Jet Airways Business                   2
Name: Airline, dtype: int64
_________________________
9/05/2019     144
12/06/2019    135
18/05/2019    129
6/06/2019     127
9/06/2019     119
21/05/2019    118
15/05/2019    106
15/06/2019    105
6/03/2019      97
21/03/2019     93
3/06/2019      92
1/06/2019      88
24/06/2019     85
1/04/2019      78
24/03/2019     78
27/06/2019     75
6/05/2019      73
24/05/2019     71
12/05/2019     68
27/05/2019     65
27/03/2019     65
1/05/2019      62
3/03/2019      62
9/03/2019      55
12/03/2019     43
18/03/2019     41
01/03/2019     34
15/03/201

In [6]:
# extracting hour and minutes from departure time and arrival time

df['Dep_Hour'] = pd.to_datetime(df['Dep_Time']).dt.hour
df['Dep_Min'] = pd.to_datetime(df['Dep_Time']).dt.minute

df['Arrival_Hour'] = pd.to_datetime(df['Arrival_Time']).dt.hour
df['Arrival_Min'] = pd.to_datetime(df['Arrival_Time']).dt.minute

In [7]:
# extracting date and month from date of journey

df['Date'] = pd.to_datetime(df['Date_of_Journey']).dt.day
df['Month'] = pd.to_datetime(df['Date_of_Journey']).dt.month

In [8]:
# retriving hours and minutes from duration

duration = list(df['Duration'])
for i in range(len(duration)):
    if len(duration[i].split()) != 2:
        if 'h' in duration[i]:
            duration[i] = duration[i].strip() + ' 0m'
        else:
            duration[i] = "0h " + duration[i]

duration_hours = []
duration_mins = []

for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep='h')[0]))
    duration_mins.append(int(duration[i].split(sep='m')[0].split()[-1]))

df['Duration_Hours'] = duration_hours
df['Duration_Mins'] = duration_mins

In [9]:
# getting day name from date

df['Date_of_Journey'] = pd.to_datetime(df['Date_of_Journey'])
df['Weekday'] = df[['Date_of_Journey']].apply(
    lambda x: dt.datetime.strftime(x['Date_of_Journey'], '%A'), axis=1)

In [10]:
# to get the different day time parts


def day_part(i):
    if (i >= 4) and (i < 8):
        return 'Early Morning'
    elif (i >= 8) and (i < 12):
        return 'Morning'
    elif (i >= 12) and (i < 16):
        return 'Noon/Afternoon'
    elif (i >= 16) and (i < 20):
        return 'Evening'
    elif (i >= 20) and (i <= 24):
        return 'Night'
    elif (i > 24) and (i < 4):
        return 'Latenight'


df['day/night'] = df['Dep_Hour'].apply(day_part)

In [11]:
df.describe()

,Dep_Hour,Dep_Min,Arrival_Hour,Arrival_Min,Date,Month,Duration_Hours,Duration_Mins
count,2671.000000,2671.000000,2671.000000,2671.000000,2671.000000,2671.000000,2671.000000,2671.000000
mean,12.603519,24.891426,13.585174,24.558218,12.026956,5.607263,10.189068,28.397604
std,5.686673,19.086522,7.040396,16.775436,8.564408,3.042952,8.390683,16.747651
min,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,0.000000,0.000000
25%,8.000000,5.000000,8.000000,10.000000,5.000000,3.000000,2.000000,15.000000
50%,12.000000,25.000000,15.000000,25.000000,6.000000,5.000000,8.000000,30.000000
75%,18.000000,45.000000,19.000000,35.000000,21.000000,6.000000,15.000000,45.000000
max,23.000000,55.000000,23.000000,55.000000,27.000000,12.000000,40.000000,55.000000


In [12]:
df[df['Duration_Hours'] > 24]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Dep_Hour,Dep_Min,Arrival_Hour,Arrival_Min,Date,Month,Duration_Hours,Duration_Mins,Weekday,day/night
10,Jet Airways,2019-03-21,Delhi,Cochin,DEL → MAA → BOM → COK,13:55,18:50 22 Mar,28h 55m,2 stops,In-flight meal not included,13,55,18,50,21,3,28,55,Thursday,Noon/Afternoon
27,Jet Airways,2019-05-21,Delhi,Cochin,DEL → BOM → COK,17:30,19:00 22 May,25h 30m,1 stop,No info,17,30,19,0,21,5,25,30,Tuesday,Evening
48,Jet Airways,2019-01-04,Kolkata,Banglore,CCU → BOM → BLR,18:55,22:35 02 Apr,27h 40m,1 stop,No info,18,55,22,35,4,1,27,40,Friday,Evening
50,Air India,2019-12-05,Kolkata,Banglore,CCU → BOM → COK → BLR,09:25,13:45 13 May,28h 20m,2 stops,No info,9,25,13,45,5,12,28,20,Thursday,Morning
52,Jet Airways,2019-03-24,Kolkata,Banglore,CCU → BOM → BLR,13:55,16:20 25 Mar,26h 25m,1 stop,No info,13,55,16,20,24,3,26,25,Sunday,Noon/Afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600,Air India,2019-01-06,Delhi,Cochin,DEL → MAA → COK,06:05,09:25 02 Jun,27h 20m,1 stop,No info,6,5,9,25,6,1,27,20,Sunday,Early Morning
2618,Jet Airways,2019-05-21,Kolkata,Banglore,CCU → DEL → BLR,17:00,22:05 22 May,29h 5m,1 stop,No info,17,0,22,5,21,5,29,5,Tuesday,Evening
2646,Jet Airways,2019-05-18,Delhi,Cochin,DEL → AMD → BOM → COK,23:05,04:25 20 May,29h 20m,2 stops,No info,23,5,4,25,18,5,29,20,Saturday,Night
2650,Air India,2019-09-05,Kolkata,Banglore,CCU → IXR → DEL → BLR,05:50,08:55 10 May,27h 5m,2 stops,No info,5,50,8,55,5,9,27,5,Thursday,Early Morning


In [13]:
# replacing hours more than 24

time=[]
for k,v in enumerate(range(24,50)):
    time.append([v,k])

time=dict(time)
time.pop(24)
time

df=df.copy()
df['Duration_Hours']=df['Duration_Hours'].replace(time)

In [14]:
df[df['Duration_Hours'] == 0]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Dep_Hour,Dep_Min,Arrival_Hour,Arrival_Min,Date,Month,Duration_Hours,Duration_Mins,Weekday,day/night
2660,Air India,2019-12-03,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,16,50,16,55,3,12,0,5,Tuesday,Evening


In [15]:
df.drop([2660], inplace=True)

In [16]:
# replacing and encoding few features

df['Destination'][df['Destination'] == 'New Delhi'] = 'Delhi'

df['Weekday'] = df['Weekday'].map({
    'Monday': 1,
    'Tuesday': 2,
    'Wednesday': 3,
    'Thursday': 4,
    'Friday': 5,
    'Saturday': 6,
    'Sunday': 7
})

df['Total_Stops'] = df['Total_Stops'].map({
    'non-stop': 0,
    '1 stop': 1,
    '2 stops': 2,
    '3 stops': 3,
    '4 stops': 4
})

df['day/night'] = df['day/night'].map({
    'Early Morning': 1,
    'Morning': 2,
    'Noon/Afternoon': 3,
    'Evening': 4,
    'Night': 5,
    'Late Night': 6
})

In [17]:
df[df['day/night'].isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Dep_Hour,Dep_Min,Arrival_Hour,Arrival_Min,Date,Month,Duration_Hours,Duration_Mins,Weekday,day/night
56,IndiGo,2019-05-18,Delhi,Cochin,DEL → BLR → COK,02:00,07:45,5h 45m,1,No info,2,0,7,45,18,5,5,45,6,NaN
62,IndiGo,2019-03-04,Delhi,Cochin,DEL → BLR → COK,02:00,07:45,5h 45m,1,No info,2,0,7,45,4,3,5,45,1,NaN
66,IndiGo,2019-06-06,Delhi,Cochin,DEL → MAA → COK,02:00,07:25,5h 25m,1,No info,2,0,7,25,6,6,5,25,4,NaN
67,Multiple carriers,2019-06-15,Delhi,Cochin,DEL → BOM → COK,03:50,12:35,8h 45m,1,No info,3,50,12,35,15,6,8,45,6,NaN
142,Jet Airways,2019-05-15,Delhi,Cochin,DEL → BOM → COK,02:15,12:35,10h 20m,1,No info,2,15,12,35,15,5,10,20,3,NaN
173,IndiGo,2019-06-04,Banglore,Delhi,BLR → DEL,01:30,04:15,2h 45m,0,No info,1,30,4,15,4,6,2,45,2,NaN
201,Jet Airways,2019-01-06,Delhi,Cochin,DEL → BOM → COK,02:15,19:00,16h 45m,1,No info,2,15,19,0,6,1,16,45,7,NaN
232,Jet Airways,2019-06-21,Mumbai,Hyderabad,BOM → HYD,02:55,04:20,1h 25m,0,No info,2,55,4,20,21,6,1,25,5,NaN
234,IndiGo,2019-05-18,Banglore,Delhi,BLR → DEL,00:30,03:25,2h 55m,0,No info,0,30,3,25,18,5,2,55,6,NaN
261,IndiGo,2019-03-27,Mumbai,Hyderabad,BOM → HYD,02:30,04:00,1h 30m,0,No info,2,30,4,0,27,3,1,30,3,NaN


In [18]:
df['day/night'] = df['day/night'].fillna(6)

In [19]:
# removing unwanted columns

df.drop(['Date_of_Journey','Route','Dep_Time','Arrival_Time','Duration','Additional_Info'],axis=1,inplace=True)

In [20]:
# saving the data

df.to_csv('preprocessed_test.csv', index=False)